## 1.Lendo arquivo CSV

In [2]:
import pandas as pd

# Carregar o CSV com os dados fornecidos
df = pd.read_csv("./dataframes/resultado-final-dataframes.csv")

# Exibir informações sobre os dados para verificar valores ausentes
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6369 entries, 0 to 6368
Data columns (total 16 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Entidade                                                         6369 non-null   object 
 1   Ano                                                              5108 non-null   float64
 2   Produção Anual de Plástico                                       69 non-null     float64
 3   Código_x                                                         2321 non-null   object 
 4   Participação na emissão global de plásticos para o oceano        2321 non-null   float64
 5   Código_y                                                         2860 non-null   object 
 6   Participação da reciclagem do lixo total regional                2860 non-null   float64
 7   Participação da queima do lixo total regio

## 2.Organizando dados
- Removendo colunas desnecessárias
- Separando Variáveis ```X``` e ```y```
- Removendo Linhas onde ```y``` contém valores ausentes

In [3]:
df_clean = df.drop(['Código_x', 'Código_y', 'Código', 'Cidade', 'Região', 
                    'Participação da reciclagem do lixo total regional', 
                    'Participação da queima do lixo total regional', 
                    'Participação do lixo descartado e mal gerido do total regional', 
                    'Participação do lixo encaminhado para aterros do total regional', 'Entidade'], axis=1)

# Separar o dataframe em variáveis independentes (X) e dependente (y)
X = df_clean.drop("Poluição da Água", axis=1)
y = df_clean["Poluição da Água"]

# Remover linhas onde y é NaN
non_nan_indices = y.dropna().index
X = X.loc[non_nan_indices]
y = y.loc[non_nan_indices]

# Verificar se ainda há valores ausentes
print(y.isnull().sum())

0


Foi necessário também usar mediana para substituir os valores ausentes em ```X```

OBS: O 'erro' abaixo, na verdade é um aviso de arredondamento, que até o momento(05/06/2024), não gerou problemas no funcionamento ou eficácia do modelo

In [4]:
from sklearn.impute import SimpleImputer

# Imputação dos valores ausentes usando a mediana
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Verificar se ainda há valores ausentes
print(pd.DataFrame(X_imputed, columns=X.columns).isnull().sum())

c:\Users\lalop\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:577: UserWarning: Skipping features without any observed values: ['Produção Anual de Plástico']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


ValueError: Shape of passed values is (6198, 4), indices imply (6198, 5)

## 3. Treinos e testes
 Divisão dos dados em conjuntos de treinamento e testes

In [5]:
from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)


## 4.Normalização dos dados
Para que todas as caracteríticas tenham uma média de 0 e desvio padrão de 1

In [6]:
from sklearn.preprocessing import StandardScaler

# Inicializar o normalizador
scaler = StandardScaler()

# Ajustar o normalizador nos dados de treinamento e transformar os dados de treinamento
X_train_scaled = scaler.fit_transform(X_train)

# Transformar os dados de teste com o normalizador ajustado
X_test_scaled = scaler.transform(X_test)



## 5.Inicializando e treinando
O regressor é inicializado e e treinado com os dados de treinamento normalizados

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Inicializar o regressor
regressor = DecisionTreeRegressor()

# Treinar o regressor com os dados de treinamento normalizados
regressor.fit(X_train_scaled, y_train)

# Fazer previsões no conjunto de teste normalizado
y_pred = regressor.predict(X_test_scaled)



## 5.Fazendo Previsão

In [9]:
test_feedback = pd.DataFrame([X.sample(n=1, random_state=None).iloc[0]], columns=X.columns)  # Pega uma amostra aleatória de X para prever

# Imputar valores ausentes na amostra selecionada
test_feedback_imputed = imputer.transform(test_feedback)

# Normalizar a amostra imputada
test_feedback_scaled = scaler.transform(test_feedback_imputed)

# Fazer a previsão
prediction = regressor.predict(test_feedback_scaled)

# Exibir o resultado
# print(f'Amostra selecionada:\n{test_feedback}\n') # caso queira ver qual amostra foi utilizada
print(f'Previsão da Poluição da Água: {prediction[0]}')

Previsão da Poluição da Água: 67.85714286


c:\Users\lalop\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:577: UserWarning: Skipping features without any observed values: ['Produção Anual de Plástico']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
